# Docker/Container Class

#### Made by: SSG McCracken

## Information

### What is a Container

### What is Docker

## Practical Excersices

#### Environment Setup Script

In [40]:
!apt-get -qq update && apt-get -q upgrade -y && apt-get -qq remove -y  docker docker-engine docker.io containerd runc && apt-get -qq install -y docker.io && echo "Done"

Reading package lists...
Building dependency tree...
Reading state information...
Calculating upgrade...
The following packages were automatically installed and are no longer required:
  libfprint-2-tod1 libllvm10 libllvm11 libllvm9 linux-headers-5.8.0-53-generic
  linux-hwe-5.8-headers-5.8.0-53 linux-image-5.8.0-53-generic
  linux-modules-5.8.0-53-generic linux-modules-extra-5.8.0-53-generic
  python3-cached-property python3-docker python3-dockerpty python3-docopt
  python3-texttable python3-websocket
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
(Reading database ... 242106 files and directories currently installed.)
Removing docker.io (20.10.7-0ubuntu1~20.04.1) ...
'/usr/share/docker.io/contrib/nuke-graph-directory.sh' -> '/var/lib/docker/nuke-graph-directory.sh'
Removing containerd (1.5.2-0ubuntu1~20.04.2) ...
Removing runc (1.0.0~rc95-0ubuntu1~20.04.2) ...
Processing triggers for man-db (2.9.1-1) ...
Preconfiguring package

In [41]:
!systemctl is-active docker

active


In [123]:
!docker stop `docker ps -aq`; docker rm `docker ps -aq`; docker rmi `docker images -aq`

"docker stop" requires at least 1 argument.
See 'docker stop --help'.

Usage:  docker stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers
"docker rm" requires at least 1 argument.
See 'docker rm --help'.

Usage:  docker rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers
Untagged: ubuntu:latest
Untagged: ubuntu@sha256:9d6a8699fb5c9c39cf08a0871bd6219f0400981c570894cd8cbea30d3424a31f
Deleted: sha256:fb52e22af1b01869e23e75089c368a1130fa538946d0411d47f964f8b1076180
Deleted: sha256:4942a1abcbfa1c325b1d7ed93d3cf6020f555be706672308a4a4a6b6d631d2e7
Untagged: alpine:latest
Untagged: alpine@sha256:e1c082e3d3c45cccac829840a25941e679c25d438cc8412c2fa221cf1a824e6a
Deleted: sha256:14119a10abf4669e8cdbdff324a9f9605d99697215a0d21c360fe8dfa8471bab
Deleted: sha256:e2eb06d8af8218cfec8210147357a68b7e13f7c485b991c288c2d01dc228bb68
Untagged: spurin/diveintoansible:portal
Untagged: spurin/diveintoansible@sha256:7836d47176865e0314caaa6ecff920f5b923daac1d5f45d105c5eb2c

___

### Docker Commands

___

#### PS

In [124]:
!docker ps -h

Flag shorthand -h has been deprecated, please use --help

Usage:  docker ps [OPTIONS]

List containers

Options:
  -a, --all             Show all containers (default shows just running)
  -f, --filter filter   Filter output based on conditions provided
      --format string   Pretty-print containers using a Go template
  -n, --last int        Show n last created containers (includes all
                        states) (default -1)
  -l, --latest          Show the latest created container (includes all
                        states)
      --no-trunc        Don't truncate output
  -q, --quiet           Only display container IDs
  -s, --size            Display total file sizes


In [125]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


>In the command above we see that we have no containers in any status running. This is because we removed all containers in our initial setup above with the command below:
> 
> ``docker stop `docker ps -aq` && docker rm `docker ps -aq` ``
>
> An example of all of these commands will will be below

> Lets start a container to show what a running container looks like in `docker ps`

In [126]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

de127a29: Pull complete 529kB/2.529kBBDigest: sha256:7d91b69e04a9029b99f3585aaaccae2baa80bcf318f4a5d2165a9898cd2dc0a1
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For 

> Below we can show all containers with `docker ps -a`

In [127]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED         STATUS                     PORTS     NAMES
22cccc58823a   hello-world   "/hello"   4 seconds ago   Exited (0) 2 seconds ago             relaxed_antonelli


> Lets start a container that will run indefinetly to show what a running container looks like with `docker ps`

In [128]:
!docker run -d ubuntu sleep infinity

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu

7b77a593: Pull complete .57MB/28.57MBBDigest: sha256:9d6a8699fb5c9c39cf08a0871bd6219f0400981c570894cd8cbea30d3424a31f
Status: Downloaded newer image for ubuntu:latest
dc4c58ea4312b60216dc4de32eef05de461a3d385819df285b1210f638be7331


> Below you will see three different commands
>
> `docker ps` shows the running container
>
> `docker ps -a` shows all containers
>
> `docker ps -aq` shows all containers but only with the container id
>
>> `docker ps -aq` is useful for piping all containers to commands like `docker rm `docker ps -aq` as you will see me use throughout this lesson

In [129]:
!docker ps

CONTAINER ID   IMAGE     COMMAND            CREATED         STATUS         PORTS     NAMES
dc4c58ea4312   ubuntu    "sleep infinity"   7 seconds ago   Up 4 seconds             crazy_ardinghelli


In [130]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND            CREATED          STATUS                      PORTS     NAMES
dc4c58ea4312   ubuntu        "sleep infinity"   8 seconds ago    Up 5 seconds                          crazy_ardinghelli
22cccc58823a   hello-world   "/hello"           18 seconds ago   Exited (0) 17 seconds ago             relaxed_antonelli


In [131]:
!docker ps -aq

dc4c58ea4312
22cccc58823a


> Below you will see me stop all containers and remove them using `docker ps -aq`

In [132]:
!docker stop `docker ps -aq` && docker rm `docker ps -aq`

dc4c58ea4312
22cccc58823a
dc4c58ea4312
22cccc58823a


> Just to verify that they are all removed we will run `docker ps -a`

In [133]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


___

#### Docker Images

In [134]:
!docker images -h

Flag shorthand -h has been deprecated, please use --help

Usage:  docker images [OPTIONS] [REPOSITORY[:TAG]]

List images

Options:
  -a, --all             Show all images (default hides intermediate images)
      --digests         Show digests
  -f, --filter filter   Filter output based on conditions provided
      --format string   Pretty-print images using a Go template
      --no-trunc        Don't truncate output
  -q, --quiet           Only show image IDs


In [135]:
!docker images -a

REPOSITORY    TAG       IMAGE ID       CREATED        SIZE
ubuntu        latest    fb52e22af1b0   8 days ago     72.8MB
hello-world   latest    d1165f221234   6 months ago   13.3kB


> Below I will use `docker -rmi <container name>` to remove a specific image

In [136]:
!docker rmi hello-world

Untagged: hello-world:latest
Untagged: hello-world@sha256:7d91b69e04a9029b99f3585aaaccae2baa80bcf318f4a5d2165a9898cd2dc0a1
Deleted: sha256:d1165f2212346b2bab48cb01c1e39ee8ad1be46b87873d9ca7a4e434980a7726
Deleted: sha256:f22b99068db93900abe17f7f5e09ec775c2826ecfe9db961fea68293744144bd


> I'll run `docker images -a` just to verify it was removed

In [137]:
!docker images -a

REPOSITORY   TAG       IMAGE ID       CREATED      SIZE
ubuntu       latest    fb52e22af1b0   8 days ago   72.8MB


> I'll use `docker pull <image name>` to pull an image to my local machine but not run it

In [138]:
!docker pull hello-world

Using default tag: latest
latest: Pulling from library/hello-world

de127a29: Pull complete 529kB/2.529kBBDigest: sha256:7d91b69e04a9029b99f3585aaaccae2baa80bcf318f4a5d2165a9898cd2dc0a1
Status: Downloaded newer image for hello-world:latest
docker.io/library/hello-world:latest


In [139]:
!docker images -a

REPOSITORY    TAG       IMAGE ID       CREATED        SIZE
ubuntu        latest    fb52e22af1b0   8 days ago     72.8MB
hello-world   latest    d1165f221234   6 months ago   13.3kB


___
#### Run

In [141]:
!docker run --help


Usage:  docker run [OPTIONS] IMAGE [COMMAND] [ARG...]

Run a command in a new container

Options:
      --add-host list                  Add a custom host-to-IP mapping
                                       (host:ip)
  -a, --attach list                    Attach to STDIN, STDOUT or STDERR
      --blkio-weight uint16            Block IO (relative weight),
                                       between 10 and 1000, or 0 to
                                       disable (default 0)
      --blkio-weight-device list       Block IO weight (relative device
                                       weight) (default [])
      --cap-add list                   Add Linux capabilities
      --cap-drop list                  Drop Linux capabilities
      --cgroup-parent string           Optional parent cgroup for the
                                       container
      --cgroupns string                Cgroup namespace to use
                                       (host|private)
                     

> Above I used `docker run -d ubuntu sleep infinity` to start a ubuntu image and execute the `sleep inifinity` command in and I used the `-d` to run it in a detached status.
> I'll do it again below

In [149]:
!docker run -d --name will_not_die ubuntu sleep infinity

bd36d6f48d692e87bdc2b9758cbf47669c9a6d2f41427cacf7c7510aa2135cb6


> So now we have a ubuntu container that has a process running. While the process is running the container will stay alive. Once the process completes the docker container wil exit

> Let me demonstrate that with the command below.
>> NOTE: This container will be ran in the "attached" mode.

In [152]:
!docker run --name will_die ubuntu sleep 5

docker: Error response from daemon: Conflict. The container name "/will_die" is already in use by container "1552a9ef0668e79248344503b9175afb6df4204fb38214093d927150c4c696b7". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [153]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND            CREATED              STATUS                          PORTS     NAMES
1552a9ef0668   ubuntu    "sleep 5"          About a minute ago   Exited (0) About a minute ago             will_die
bd36d6f48d69   ubuntu    "sleep infinity"   About a minute ago   Up About a minute                         will_not_die


> Above you can see that the "will_die" container exited, while the "will_not_die" is still running

> We can attach to the container by attaching to it using `docker attach will_not_die` or `docker attach <container_id>`

> We can map ports using `-p host_port:container_port`

In [164]:
!docker run --name port_test -d --rm -p 8080:80 yeasy/simple-web

7d369c03d1e26892fb8fefaa319b7b09aa7cbfd7cec6265823dd2e65a1f7ce42


In [166]:
!docker stop port_test

Error response from daemon: No such container: port_test


___
#### EXEC

> We can also run a command on a container by using `docker exec`

In [159]:
!docker exec will_not_die env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=bd36d6f48d69
HOME=/root


> We can also use this to get a terminal to the container using `docker exec -it <name> /bin/bash`